# Bitcoin API Demo – Using Azure SDK and Utility Wrappers

This notebook demonstrates how to use the modular API provided in `bitcoin_utils.py` to:

- Fetch real-time Bitcoin prices from CoinGecko
- Authenticate with Azure using synchronous and asynchronous credentials
- Upload event data to Azure Blob Storage

All core logic is imported from the utility module, allowing us to keep this notebook clean and focused on usage.


### Imports and Setup

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from azure.storage.blob.aio import BlobServiceClient

from bitcoin_utils import (
    fetch_bitcoin_price_usd,
    get_azure_sync_credential,
    get_azure_async_credential,
    upload_buffer_to_blob
)


### Loading Environment Variables

In [2]:
# Load .env file so we can access credentials and config
load_dotenv()

True

### Fetching Bitcoin Price (Sync)

In [3]:
btc_price = fetch_bitcoin_price_usd()

if btc_price is not None:
    print(f"Current BTC price (USD): {btc_price}")
else:
    print("Failed to fetch Bitcoin price.")


Current BTC price (USD): 103852.0


### Getting Azure Credentials

In [4]:
# Get a synchronous Azure credential (for use with EventHubProducerClient)
sync_credential = get_azure_sync_credential()

# Get an asynchronous Azure credential (for use with EventHubConsumerClient or Blob Storage)
async_credential = get_azure_async_credential()

print("Azure credentials loaded.")


Azure credentials loaded.


### Preparing Dummy Event Data

In [5]:
import json
import time

dummy_events = [
    {
        "currency": "BTC",
        "price_usd": 103000.0,
        "timestamp": time.time()
    },
    {
        "currency": "BTC",
        "price_usd": 103010.5,
        "timestamp": time.time()
    },
    {
        "currency": "BTC",
        "price_usd": 103020.8,
        "timestamp": time.time()
    }
]

print("Prepared dummy event records for upload.")


Prepared dummy event records for upload.


###  Uploading to Azure Blob Storage (Async)

In [6]:
# Define blob container and account URL from environment
container_name = os.getenv("STORAGE_CONTAINER_NAME")
account_url = os.getenv("STORAGE_ACCOUNT_URL")

# Ensure both are loaded
assert container_name and account_url, "Missing storage config in .env"

# Upload using the async wrapper
async def test_upload():
    blob_client = BlobServiceClient(account_url=account_url, credential=async_credential)
    await upload_buffer_to_blob(blob_client, container_name, dummy_events)

# Run the upload
await test_upload()


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000282662BD4F0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x0000028266543470>, 5485.281)])']
connector: <aiohttp.connector.TCPConnector object at 0x0000028266523CE0>


## Conclusion 

In this notebook, we demonstrated how to:
- Use the API wrapper functions from `bitcoin_utils.py`
- Fetch live Bitcoin data using CoinGecko
- Authenticate with Azure securely via environment variables
- Upload event records to Azure Blob Storage asynchronously

By modularizing all logic into a utility module, we made this notebook easy to understand and beginner-friendly.

You can now integrate these APIs into your own data engineering or real-time analytics pipelines.
